In [2]:
import git
import shutil
import os
import requests
import pandas as pd



In [2]:
def check_contents(repo_location):
    """
    Checks the contents of the repository.

    Args:
        repo_location (str): Full repository URL.

    Returns:
        bool: True if the repository contains 'splitter.py'.
    """
    owner = 'learn-co-curriculum'
    repo_name = repo_location.split('/')[-1]
    api_url = f'https://api.github.com/repos/{owner}/{repo_name}/contents?ref=curriculum'
    print(f'Checking contents in {api_url}')
    MY_TOKEN = os.getenv('GITHUB_TOKEN')
    headers = {'Authorization': f'Bearer {MY_TOKEN}'}
    response = requests.get(api_url, headers=headers).json()
    list_of_contents = []
    for item in response:
        list_of_contents.append(item['name'])
    return 'splitter.py' in list_of_contents

In [3]:
def check_branch(repo_location):
    """Checks the repository for a curriculum branch which is necessary for the github actions to function

    Args:
        repo_location (URL): full repository URL

    Returns:
        bool: True if the repository contains a curriculum branch
    """
    owner = 'learn-co-curriculum'
    repo_name = repo_location.split('/')[-1]
    api_url = f'https://api.github.com/repos/{owner}/{repo_name}/branches'
    print(f'Checking branches in {api_url}')
    MY_TOKEN = os.getenv('GITHUB_TOKEN')
    headers = {'Authorization': f'Bearer {MY_TOKEN}'}
    response = requests.get(api_url, headers=headers).json()
    branch_list = []
    for branch in response:
        branch_list.append(branch['name'])
        
    return 'curriculum' in branch_list

In [4]:
def add_files_to_repo(repo_location):
    """
    Adds files to the repository.

    Args:
        repo_location (str): Full repository URL.

    Raises:
        ValueError: If the repository URL is invalid.
    """
        
    if check_branch(repo_location):
        owner = 'learn-co-curriculum'
        repo_name = repo_location.split('/')[-1]
        api_url = f'https://api.github.com/repos/{owner}/{repo_name}'
        print(f'Working on {api_url}')
        MY_TOKEN = os.getenv('GITHUB_TOKEN')
        headers = {'Authorization': f'Bearer {MY_TOKEN}'}
        response = requests.get(api_url, headers=headers).json()
        repo_path = os.path.join(os.getcwd(), repo_name)
        workflows_path = os.path.join(repo_path, '.github', 'workflows')
        if check_contents(repo_location):
            print('This repo already has the actions files')
        
        else:
            if 'ssh_url' in response.keys():
                repo = git.Repo.clone_from(response['ssh_url'], repo_path)
                repo.git.checkout('curriculum')
                os.makedirs(workflows_path, exist_ok=True)
                yaml_file = 'branch_split.yml'
                script_file = 'splitter.py'
                yaml_destination = os.path.join(repo_path, '.github', 'workflows', 'branch_split.yml')
                script_destination = os.path.join(repo_path, 'splitter.py')
                shutil.copyfile(yaml_file, yaml_destination)
                shutil.copyfile(script_file, script_destination)
                repo.git.add(".")
                repo.git.commit(m="Adding files via FIS actions updater script")
                repo.git.push()
                print(f'Added files to {repo_location}.')
                shutil.rmtree(repo_path) 
                print('Cleaning up directory tree')
            else:
                print(f'{api_url} is not a valid url')            
    
    else:
        print('This repo has no curriculum branch')



## Run all above cells to load all necessary dependencies

Replace the URL in the `add_files_to_repo` function with the URL you wish to update with the actions files.

In [ ]:
"Replace the URL below with the URL you wish to add files to"

add_files_to_repo("https://github.com/learn-co-curriculum/dsc-structure-of-this-course")

In [ ]:
"The script below will parse a list of phases and add the necessary files to each repository"
for phase, course in list_of_phases:
    os.system(f"github-to-canvas --query {course} > phase_{phase}_canvas.yml")
    os.system(f"github-to-canvas --map phase_{phase}_canvas.yml --urls-only > phase_{phase}_canvas.csv")
    df = pd.read_csv(f'phase_{phase}_canvas.csv')
    df.columns = ['url']
    
    completed = []
    for i, item in df.iterrows():
        add_files_to_repo(item['url'])
        completed.append(i)
    df = df.drop(df.index[completed])